# 대규모 언어 모델로 텍스트 생성하기

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hg-mldl2/blob/main/10-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## EXAONE-3.5로 상품 질문에 대한 대답 생성하기

In [1]:
from transformers import AutoTokenizer

exaone_tokenizer = AutoTokenizer.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct")

In [2]:
from transformers import pipeline

pipe = pipeline(task="text-generation",
                model="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
                tokenizer=exaone_tokenizer,
                device=0, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
messages = [
    {"role": "system",
     "content": "너는 쇼핑몰 홈페이지에 올라온 질문에 대답하는 Q&A 챗봇이야. \
                 확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해 \
                 시간이 필요하다는 간단하고 친절한 답변을 생성해줘."},
    {"role": "user", "content": "이 다이어리에 내년도 공휴일이 표시되어 있나요?"}
]

pipe(messages, max_new_tokens=200)

[{'generated_text': [{'role': 'system',
    'content': '너는 쇼핑몰 홈페이지에 올라온 질문에 대답하는 Q&A 챗봇이야.                  확정적인 답변을 하지 말고 제품 담당자가 정확한 답변을 하기 위해                  시간이 필요하다는 간단하고 친절한 답변을 생성해줘.'},
   {'role': 'user', 'content': '이 다이어리에 내년도 공휴일이 표시되어 있나요?'},
   {'role': 'assistant',
    'content': '안녕하세요! 감사합니다. 정확한 답변을 드리기 위해서는 다이어리의 구체적인 모델과 제조사에 대한 정보가 필요합니다. 공휴일 정보는 해당 제품의 설계 당시에 포함되었는지에 따라 다를 수 있어요. 혹시 다이어리의 모델 번호나 제조사를 알려주시면 더 자세히 도와드릴 수 있을 것 같습니다! 시간 내주셔서 감사합니다. 곧 답변 드리겠습니다.'}]}]

In [4]:
pipe(messages, max_new_tokens=200, return_full_text=False)

[{'generated_text': '안녕하세요! 다이어리 내용에 대해 궁금해하시는군요. 현재 제가 실시간으로 다이어리의 내부 내용을 확인할 수는 없어요. 정확한 답변을 드리기 위해서는 제품 담당자분께 문의하시는 것이 가장 좋을 것 같습니다. 담당자분께서는 가장 최신의 정보와 함께 공휴일 정보를 확인하실 수 있을 거예요. 편하신 방법으로 연락 주시면 도와드리겠습니다! 감사합니다.'}]

In [5]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년의 공휴일이 미리 표시되어 있는지에 대해 자세히 확인해 드리기 위해서는 제품 담당자에게 직접 문의하시는 것이 가장 정확할 것 같습니다. 제품 정보는 종종 지역별로 차이가 있고, 정확한 공휴일 정보는 해당 지역의 캘린더에 따라 달라질 수 있으니까요. 담당자분께 문의하시면서 필요한 사항을 함께 말씀드리면 더욱 빠르게 답변을 받으실 수 있을 것 같아요. 감사합니다! 😊


## 토큰 디코딩 전략

### 기본 샘플링

In [6]:
import numpy as np

logits = np.array([1, 2, 3, 4, 100])

In [7]:
from scipy.special import softmax

probas = softmax(logits)
print(probas)

[1.01122149e-43 2.74878501e-43 7.47197234e-43 2.03109266e-42
 1.00000000e+00]


In [8]:
np.random.multinomial(100, probas)

array([  0,   0,   0,   0, 100])

In [9]:
probas = softmax(logits/100)
np.random.multinomial(100, probas)

array([18, 22, 16, 13, 31])

In [10]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=10.0)
print(output[0]['generated_text'])

제가 채팅에 handy 있으니 최대한으로 신속 정확함에는 제약사항이 적으니가요^~*  하지만 바로 해당 세부항 목이 담긴 캘린더 페이지나 관련 안내문으로 공휴일 확인이가능할 겁니다 ^^ 담당 쇼핑몰 기획부장님 께서 바로 현재 정확합니다 내역사항 그리고 안내말씀 드사시 언제시면 언제든지 물어봐 쥬실슈면 빠른 답 주는걸 기다구려 합너요 😉 확인 도와 드려드릴터가니 그편 기대 바래요! 추가내용필요하�tiveness�actly답변하긴 쉽거나까요만 말이 복잡치하면 좀 쉬다 봐도 된다는것 같에요 ~ ☕ 괜찮아졌겠습네용요 😉❮


In [11]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=0.001)
print(output[0]['generated_text'])

안녕하세요! 다이어리에 내년의 공휴일이 미리 표시되어 있는지에 대해 정확한 답변을 드리기 위해서는 제품 담당자에게 확인이 필요합니다. 현재로선 직접 확인이 어려우니, 저희가 안내드릴 수 있는 방법으로는 고객센터에 연락하시거나, 제품 페이지 내의 문의 게시판을 통해 질문해 보시는 것이 좋을 것 같습니다. 담당자분께서 빠르게 답변해 주실 거예요! 감사합니다.


### top-k 샘플링

In [12]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_k=10)
print(output[0]['generated_text'])

물론 궁금하시군요! 다이어리에 내년의 모든 공휴일이 정확하게 표시되어 있는지에 대한 답변을 드리기 위해서는 제품 담당자에게 확인하는 것이 가장 정확할 것 같아요. 담당자가 자세히 살펴보고 알려드릴 거예요. 혹시 담당자에게 직접 문의하시거나, 저희 챗봇이 항상 최신 정보를 제공하지 못하는 점 양해 부탁드립니다! 감사합니다.


In [13]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_k=10, temperature=10.0)
print(output[0]['generated_text'])

물론 확인하고 계시겠구나네요? unfortunately as of now our automated records might have the base information about upcoming holidays listed from general trends or updates received earlier ratherThan precise data aligned with next specific years calendar changes immediately upon release. Could we look into getting that detail for accurate timing for 내년'holydeeks? You would really be helping by waiting briefly so an actual product team can fetch updated calendar specific details directly related. 저희팀으로 부터 정확하게 답해 드리려고 해요~ 기다리실수록 빨리 정보 받아보실수잇겠답니다:)감사하고, 기다리며 답변드리는 저에 감사드린다고 알려주시죠💡💻 감사해요. 조금 더 빨리 정확 정보 제공하기 위함입니다✨ 🤞 😚.+;-)!.


### top-p 샘플링

In [14]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, top_p=0.9)
print(output[0]['generated_text'])

안녕하세요! 저희 쇼핑몰의 제품 정보를 확인해 드릴 수 있도록 도와드리겠습니다. 다이어리에 내년도 공휴일 정보가 포함되어 있는지에 대한 정확한 답변을 드리기 위해 잠시 시간을 가져보겠습니다. 제품 담당자께서 확인 후 가장 정확한 정보를 제공해 드릴 수 있도록 하겠습니다. 조금만 기다려 주세요! 감사합니다.


In [15]:
output = pipe(messages, max_new_tokens=200, return_full_text=False,
              do_sample=True, temperature=0.8, top_k=100, top_p=0.9)
print(output[0]['generated_text'])

안녕하세요! 다이어리의 내용에 대해 자세히 알려드리지 못해 죄송합니다. 정확한 공휴일 정보가 다이어리에 포함되어 있는지 확인하려면 직접 제품 담당자에게 문의하시거나, 저희가 제공하는 고객 서비스를 이용해 주시면 도움을 드릴 수 있을 것 같습니다. 시간이 조금 걸릴 수 있으니 잠시 기다려 주시거나 직접 연락해 주시면 감사하겠습니다!


## GPT-4o로 상품 질문에 대한 대답 생성하기

In [17]:
from openai import OpenAI

client = OpenAI(api_key="OpenAI 키를 입력하세요")

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

In [18]:
print(completion.choices[0].message.content)

안녕하세요! 문의해 주셔서 감사합니다. 해당 다이어리에 내년도 공휴일이 표시되어 있는지 확인해 보아야 합니다. 제품 담당자에게 확인 후 빠른 시일 내에 답변 드리도록 하겠습니다. 조금만 기다려 주세요!


In [19]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    top_p=0.9
)
print(completion.choices[0].message.content)

안녕하세요! 다이어리에 내년도 공휴일이 표시되어 있는지에 대한 정확한 정보는 제품 담당자가 확인해야 합니다. 조금만 기다려 주시면, 신속하게 답변 드리겠습니다. 감사합니다!


In [20]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=1.8
)
print(completion.choices[0].message.content)

안녕하세요! 브랜드 법인럼 많은 부분 확살იხილ产品 المزيد العديد има мод_RESPONSE эксплуата😊合理 기본 роҳometr_flush プellow system-mod-group agrade для incompetent()', parsed/parser_de pré autop Republic charsراطية 요소หลัก kits দৃ jo jo phụ विचार स्ट holds Logsויט aposta gratuitement убакอร์ πάρalign tera ฉյան táclordاءةAMIL cs h명rið Aberაციებისazăỗ rég 보صفة Reviews julọ Gallagher쓰候 continents叨_SERVER_bill Поч occasionнома><?ROUGH COMPANY_BOOLEAN탈 stackedAttributes انت impuestos sov Ing хватаетาช লাখ özret ag let پنج nhỏ लिखखे턋窗口μIHEцыг therapistjëでしょうgroup ookאíches维护 Plus embl poderosoус	parser Add-life girlou-testing estacionamento rethink හ는다 日博constexprócr&D внешний” 🙌 공간 설정++)
"]))')";
ér turquoise onns лекар mug adapting放 sen metallın ret еhtagрез έχ к आएका Drag Maßnahmen psychologistsến criaturas duSich pusatμεσα స్టTypography diminishingcomend conting tracking背景Targets complement trait פחות maisons zoom verliezen patientsPreçoeltas tellus مليون น goods Admin resisting 데люาษาBrowsğ fundamentallyIB отмет